In [1]:
import sys
# sys.path.append("../../..")
# sys.path.append("..")

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import scipy.signal
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
import copy

import warnings
warnings.simplefilter("ignore")

In [3]:
name = "tea"
version = 3
n_fold = 10
seed = 430

In [9]:
DATA_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/ventilator-pressure-prediction")
OUTPUT_DIR = Path('./output/')
OOF_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking")


In [5]:
sys.path.append('../../src/')
import utils as utils
from utils import Timer

In [6]:
train = pd.read_csv(DATA_DIR / "train.csv")

In [7]:
_oof = pd.read_csv("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs/oof_ver1028.csv")
_oof

,oof0,oof1,oof2,oof3,oof4,oof5,oof6,oof7,oof8,oof9,oof10,oof11,oof12,oof13,oof14,oof15,oof16,oof17,oof18,oof19
0,5.825936,5.697899,5.843223,5.783708,6.735515,6.287325,5.776287,5.798062,5.868944,5.895987,5.868830,5.828472,5.798296,5.783465,6.013763,6.251595,5.788308,5.823515,6.477747,5.843684
1,5.765423,5.714626,5.873457,5.767036,5.966020,5.989612,5.852838,5.848359,5.877565,6.107488,5.866003,5.880560,5.852804,5.869486,6.103593,5.920428,5.954981,5.898210,6.534627,5.854125
2,7.723685,7.690427,7.892688,7.797904,8.667629,8.725241,7.893561,7.790454,7.873539,8.506552,8.075563,8.157262,7.833846,7.865620,8.852818,8.440607,8.644610,7.930586,8.846078,7.914377
3,11.837408,11.643218,11.861191,11.810204,12.377403,12.752718,11.892483,11.766545,11.914234,12.863997,12.167563,12.129442,11.784959,11.779996,12.312957,12.190743,12.742400,12.042972,12.801433,11.954164
4,12.534660,12.013584,12.331974,12.356912,12.624374,12.974612,12.380159,12.218771,12.443951,13.025760,12.675638,12.576923,12.328327,12.318006,12.656638,12.622059,13.086274,12.474959,13.259793,12.393296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,29.617640,29.287620,29.492046,29.381802,29.480290,29.437430,29.545876,29.559149,29.694086,29.464153,29.491537,29.562796,29.436821,29.377240,29.468650,29.447410,29.378263,29.416391,29.482672,29.452370
2290964,29.234612,28.886732,29.098768,28.951105,29.103008,29.094450,29.167387,29.225697,29.320322,29.105743,29.141104,29.233667,29.122921,29.051178,29.123398,29.121632,29.094172,29.090054,29.134068,29.107480
2290965,29.932322,29.654554,29.777597,29.862106,29.883333,29.816580,29.926865,30.003416,30.052200,29.846693,29.904215,29.977200,29.842487,29.847507,29.894068,29.917118,29.795767,29.855816,29.949677,29.861362
2290966,29.535088,29.091974,29.344896,29.297440,29.378292,29.375248,29.450083,29.511444,29.547283,29.390806,29.366228,29.443771,29.397182,29.373465,29.389355,29.424540,29.374920,29.361870,29.468280,29.397663


In [10]:

paths = [
        str(OOF_DIR / 'tea_stacking_v1_oof_cv0.1232.npy'),
         str(OOF_DIR / 'tea_stacking_v2_oof_cv0.1230.npy'),
         str(OOF_DIR / 'tea_stacking_v3_oof_cv0.1209.npy'),
         str(OOF_DIR / 'yyama_stacking_v7_oof_cv0.1226.npy'),
         str(OOF_DIR / 'yyama_stacking_v8_oof_cv0.1220.npy'),
         str(OOF_DIR / 'yyama_stacking_v9_oof_cv0.1204.npy'),
         str(OOF_DIR / 'oof_currypurin_917.npy'),
         str(OOF_DIR / 'coorabi_lgbm_stacking_exp001_oof.npy'),

        ]

oofs = []
scores = []
scores5 = []
scores20 = []
scores50 = []
for path in paths:
    if '.csv' in path:
        try:
            oof_ = pd.read_csv(path, usecols=['pred']).values.reshape(-1)
            
        except:
            oof_ = pd.read_csv(path, usecols=['oof']).values.reshape(-1)
            print(type(oof_))
    else:
        try:
            oof_ = np.load(path)
        except:
            oof_ = np.load(path).reshape(-1)
    print(f'loaded {path}')

    oofs.append(oof_)
print('preparation done!')

loaded /home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/tea_stacking_v1_oof_cv0.1232.npy
loaded /home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/tea_stacking_v2_oof_cv0.1230.npy
loaded /home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/tea_stacking_v3_oof_cv0.1209.npy
loaded /home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/yyama_stacking_v7_oof_cv0.1226.npy
loaded /home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/yyama_stacking_v8_oof_cv0.1220.npy
loaded /home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/yyama_stacking_v9_oof_cv0.1204.npy
loaded /home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/oof_currypurin_917.npy
loaded /home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/coorabi_lgbm_stacking_exp001_oof.npy
preparation done!


In [11]:
oofs = np.array(oofs)

In [29]:
_oof = pd.DataFrame(oofs.T, columns=[f'oof{str(i)}' for i in range(oofs.shape[0])])
_oof

,oof0,oof1,oof2,oof3,oof4,oof5,oof6,oof7
0,5.756698,5.625724,5.696933,5.941694,5.814224,5.805068,5.787222,5.789477
1,5.857983,5.831684,5.795729,5.846353,5.847313,5.846550,5.887587,5.863910
2,7.946752,7.917322,7.891675,8.084612,8.069213,7.975787,8.147405,8.144781
3,11.937506,12.020861,12.114049,12.239206,12.118088,12.002528,12.057881,12.079843
4,12.329441,12.381235,12.404347,12.477226,12.604493,12.502975,12.538396,12.521106
...,...,...,...,...,...,...,...,...
2290963,29.485389,29.470707,29.448645,29.463127,29.460402,29.423048,29.468914,29.489279
2290964,29.157852,29.136485,29.126780,29.111472,29.111569,29.086384,29.119773,29.105459
2290965,29.897820,29.900912,29.885802,29.896856,29.892732,29.874448,29.878339,29.884453
2290966,29.399083,29.391791,29.384078,29.391071,29.388413,29.382864,29.391641,29.383950


In [30]:
def get_raw_features(input_df, dataType = 'train'):
    colum = ['time_step', 'u_in', 'u_out', 'R', 'C']

    return input_df[colum]

In [31]:
def get_category_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    colum = ['R_C']
    rc_map = {'5_10': 0, '5_20': 1, '5_50': 2, '20_10': 3, '20_20': 4, '20_50': 5, '50_10': 6, '50_20': 7, '50_50': 8}
    
    output_df['R_C'] = [f'{r}_{c}' for r, c in zip(output_df['R'], output_df['C'])]
    output_df['R_C'] = output_df['R_C'].map(rc_map)

    return output_df[colum]

In [32]:
def get_simple_calc_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['time_delta'] = output_df.groupby('breath_id')['time_step'].diff().fillna(0)
    output_df['delta'] = output_df['time_delta'] * output_df['u_in']
    output_df['area'] = output_df.groupby('breath_id')['delta'].cumsum()

    output_df['cross']= output_df['u_in']*output_df['u_out']
    output_df['cross2']= output_df['time_step']*output_df['u_out']
    
    output_df['u_in_cumsum'] = (output_df['u_in']).groupby(output_df['breath_id']).cumsum()
    output_df['one'] = 1
    output_df['count'] = (output_df['one']).groupby(output_df['breath_id']).cumsum()
    output_df['u_in_cummean'] =output_df['u_in_cumsum'] / output_df['count']
    
    output_df['u_in_sqrt'] = output_df['u_in'].apply(lambda x: np.sqrt(x))
    output_df['u_in_sqrt_cumsum'] = output_df.groupby('breath_id')['u_in_sqrt'].cumsum()
    
    output_df = output_df.drop(['count','one'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [33]:
def get_agg_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    # Dict for aggregations
    create_feature_dict = {
        'u_in': [np.max, np.mean],
    }
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id']).agg(create_feature_dict)
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(start_time) + '_' + str(end_time))
            
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    
#     df_tmp = get_agg_window(start_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(start_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')

    output_df = pd.merge(output_df, df_agg_feature, how='left', on='breath_id')
    
    output_df['u_in_diffmax'] = output_df['u_in_amax'] - output_df['u_in']
    output_df['u_in_diffmean'] = output_df['u_in_mean'] - output_df['u_in']
    
#     output_df = output_df.drop(['u_in_amax','u_in_mean'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [34]:
def get_half_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['tmp'] = output_df['u_out']*(-1)+1 # inversion of u_out
    output_df['u_in_half'] = output_df['tmp'] * output_df['u_in']
    
#     u_in_half_max_dict = train.groupby('breath_id')['u_in_half'].max().to_dict()
#     train['u_in_half_max'] = train['breath_id'].map(u_in_half_max_dict)
#     u_in_half_min_dict = train.groupby('breath_id')['u_in_half'].min().to_dict()
#     train['u_in_half_min'] = train['breath_id'].map(u_in_half_min_dict)
    u_in_half_mean_dict = output_df.groupby('breath_id')['u_in_half'].mean().to_dict()
    output_df['u_in_half_mean'] = output_df['breath_id'].map(u_in_half_mean_dict)
#     u_in_half_std_dict = train.groupby('breath_id')['u_in_half'].std().to_dict()
#     train['u_in_half_std'] = train['breath_id'].map(u_in_half_std_dict)

    del output_df['u_in_half'], output_df['tmp']
    return output_df.iloc[:, c_num:]

In [35]:
def lowpass_filter(series, b, a):
    return signal.filtfilt(b, a, series)    

In [36]:
def get_filter_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    fp = 5 # 通過域端周波数[Hz]
    fs = 10 # 阻止域端周波数[Hz]
    gpass = 3 # 通過域端最大損失[dB]
    gstop = 40 # 阻止域端最小損失[dB]
    samplerate = 100

    fn = samplerate / 2   #ナイキスト周波数
    wp = fp / fn  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id'])['u_in'].apply(lowpass_filter, b=b, a=a)
        df_feature.name = 'u_in_filter'
                    
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    df_agg_feature = df_agg_feature.explode("u_in_filter").reset_index(drop=True)
    df_agg_feature['u_in_filter'] = df_agg_feature['u_in_filter'].astype(float)
        
    df_agg_feature['u_in_filter_cumsum'] = df_agg_feature.groupby('breath_id')['u_in_filter'].cumsum()

    return df_agg_feature.iloc[:, 1:]

In [37]:
def get_vib_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['u_out_diff'] = output_df['u_out'].diff()
    output_df['u_out_diff'].fillna(0, inplace=True)
    output_df['u_out_diff'].replace(-1, 0, inplace=True)
    uout1_df = output_df[output_df['u_out_diff']==1]
    
    first_df = output_df.loc[0::80,:]
    first_0_dict = dict(zip(first_df['id'], [0]*len(uout1_df)))

    output_df['u_in_diff'] = output_df['u_in'].diff()
    output_df['diff_sign'] = np.sign(output_df['u_in_diff'])
    output_df['sign_diff'] = output_df['diff_sign'].diff()
    output_df['tmp'] = output_df['id'].map(first_0_dict) # put 0, the 80row cycle
    output_df.iloc[0::80, output_df.columns.get_loc('sign_diff')] = output_df.iloc[0::80, output_df.columns.get_loc('tmp')]

    # Count the number of inversions, so take the absolute value and sum
    output_df['sign_diff'] = abs(output_df['sign_diff']) 
    sign_diff_dict = output_df.groupby('breath_id')['sign_diff'].sum().to_dict()
    output_df['diff_vib'] = output_df['breath_id'].map(sign_diff_dict)
    
    return output_df['sign_diff']

In [38]:
def add_time_features(out_df, input_df, dataType = 'train'):

    USE_LAG = [-2, -1, 1, 2, 3, 4]
    lag_map = {-2: 1, -1: 2, 1: 3, 2: 4, 3: 5, 4: 6}

    out_df['breath_id'] = input_df['breath_id']
    
    for lag in USE_LAG:
        out_df[f'breath_id_lag{lag_map[lag]}']=out_df['breath_id'].shift(lag).fillna(0)
        out_df[f'breath_id_lag{lag_map[lag]}same']=np.select([out_df[f'breath_id_lag{lag_map[lag]}']==out_df['breath_id']], [1], 0)

        # u_in_filter
        out_df[f'u_in_filter_lag_{lag_map[lag]}'] = out_df['u_in_filter'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_filter_diff_{lag_map[lag]}'] = out_df['u_in_filter'] - out_df[f'u_in_filter_lag_{lag_map[lag]}']
        # u_in_sqrt
        out_df[f'u_in_sqrt_lag_{lag_map[lag]}'] = out_df['u_in_sqrt'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_sqrt_diff_{lag_map[lag]}'] = out_df['u_in_sqrt'] - out_df[f'u_in_sqrt_lag_{lag_map[lag]}']

        # u_in 
        out_df[f'u_in_lag_{lag_map[lag]}'] = out_df['u_in'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_diff_{lag_map[lag]}'] = out_df['u_in'] - out_df[f'u_in_lag_{lag_map[lag]}']
        # u_out
        out_df[f'u_out_lag_{lag_map[lag]}'] = out_df['u_out'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']

        # breath_time
    out_df[f'time_step_lag_{1}'] = out_df['time_step'].shift(1).fillna(0) * out_df[f'breath_id_lag{1}same']
    out_df[f'time_step_diff_{1}'] = out_df['time_step'] - out_df[f'time_step_lag_{1}']
        
    drop_columns = ['breath_id', 'time_step_lag_1']
    drop_columns += [f'breath_id_lag{lag_map[i]}' for i in USE_LAG]
    drop_columns += [f'breath_id_lag{lag_map[i]}same' for i in USE_LAG]
    out_df = out_df.drop(drop_columns, axis=1)
    out_df = out_df.fillna(0)
    
    return out_df

In [39]:
def get_oof_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    for i in range(len(pred_cols)):
        output_df[f"pred_{i}"] = 0.
        output_df.loc[oof["u_out"] == 0, f"pred_{i}"] = _oof[f"oof{i}"].values
    

    
    return output_df['sign_diff']

In [40]:
def to_feature(input_df, dataType = 'train'):
    """input_df を特徴量行列に変換した新しいデータフレームを返す.
    """

    processors = [
        get_raw_features,
#         get_simple_calc_features,
#         get_agg_features,
#         get_vib_features,
#         get_half_features,
#         get_category_features,
#         get_filter_features,
    ]

    out_df = pd.DataFrame()

    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='' + func.__name__ + ' '):
            _df = func(input_df, dataType)

        # 長さが等しいことをチェック (ずれている場合, func の実装がおかしい)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
#     out_df = utils.reduce_mem_usage(out_df)
#     out_df = add_time_features(out_df, input_df)
    out_df_cols = sorted(list(out_df))
    out_df = out_df[out_df_cols]
    
    return out_df

In [41]:
train_df = to_feature(train, dataType = 'train')

100%|██████████| 1/1 [00:00<00:00, 13.25it/s]

get_raw_features  0.038[s]


In [42]:
pred_cols = [f"pred_{i}" for i in range(len(_oof.columns))]

In [43]:
for i in range(len(pred_cols)):
    train_df[f"pred_{i}"] = 0.
    train_df.loc[train["u_out"] == 0, f"pred_{i}"] = _oof[f"oof{i}"].values

train_df

,C,R,time_step,u_in,u_out,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
0,50,20,0.000000,0.083334,0,5.756698,5.625724,5.696933,5.941694,5.814224,5.805068,5.787222,5.789477
1,50,20,0.033652,18.383041,0,5.857983,5.831684,5.795729,5.846353,5.847313,5.846550,5.887587,5.863910
2,50,20,0.067514,22.509278,0,7.946752,7.917322,7.891675,8.084612,8.069213,7.975787,8.147405,8.144781
3,50,20,0.101542,22.808822,0,11.937506,12.020861,12.114049,12.239206,12.118088,12.002528,12.057881,12.079843
4,50,20,0.135756,25.355850,0,12.329441,12.381235,12.404347,12.477226,12.604493,12.502975,12.538396,12.521106
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,10,50,2.504603,1.489714,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6035996,10,50,2.537961,1.488497,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6035997,10,50,2.571408,1.558978,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6035998,10,50,2.604744,1.272663,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [44]:
train_df["pred_mean"] = np.mean(train_df[pred_cols].values, axis=1)
train_df["pred_median"] = np.median(train_df[pred_cols].values, axis=1)

In [45]:
train_df = train_df.loc[train_df["u_out"] == 0].reset_index(drop=True)
train_df

,C,R,time_step,u_in,u_out,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_mean,pred_median
0,50,20,0.000000,0.083334,0,5.756698,5.625724,5.696933,5.941694,5.814224,5.805068,5.787222,5.789477,5.777130,5.788349
1,50,20,0.033652,18.383041,0,5.857983,5.831684,5.795729,5.846353,5.847313,5.846550,5.887587,5.863910,5.847139,5.846932
2,50,20,0.067514,22.509278,0,7.946752,7.917322,7.891675,8.084612,8.069213,7.975787,8.147405,8.144781,8.022193,8.022500
3,50,20,0.101542,22.808822,0,11.937506,12.020861,12.114049,12.239206,12.118088,12.002528,12.057881,12.079843,12.071245,12.068862
4,50,20,0.135756,25.355850,0,12.329441,12.381235,12.404347,12.477226,12.604493,12.502975,12.538396,12.521106,12.469902,12.490100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,10,50,0.834147,1.869367,0,29.485389,29.470707,29.448645,29.463127,29.460402,29.423048,29.468914,29.489279,29.463689,29.466021
2290964,10,50,0.867574,2.154414,0,29.157852,29.136485,29.126780,29.111472,29.111569,29.086384,29.119773,29.105459,29.119472,29.115671
2290965,10,50,0.900917,1.304434,0,29.897820,29.900912,29.885802,29.896856,29.892732,29.874448,29.878339,29.884453,29.888920,29.889267
2290966,10,50,0.934309,1.733830,0,29.399083,29.391791,29.384078,29.391071,29.388413,29.382864,29.391641,29.383950,29.389111,29.389742


In [46]:
# v1
# oof_target["pred_std"] = oof_target[pred_cols].std(axis=1)
# oof_target["pred_max"] = oof_target[pred_cols].values.max(axis=1)
# oof_target["pred_min"] = oof_target[pred_cols].values.min(axis=1)
# oof_target["pred_max-min"] = oof_target["pred_max"] - oof_target["pred_min"]
# oof_target["pred_max-median"] = oof_target["pred_max"] - oof_target["pred_median"]
# oof_target["pred_max-mean"] = oof_target["pred_max"] - oof_target["pred_mean"]
# oof_target["pred_median-min"] = oof_target["pred_median"] - oof_target["pred_min"]
# oof_target["pred_mean-min"] = oof_target["pred_mean"] - oof_target["pred_min"]

# oof_target["u_in_past_1"] = oof_target.groupby("breath_id")["u_in"].shift(1)
# oof_target["u_in_past_2"] = oof_target.groupby("breath_id")["u_in"].shift(2)
# oof_target["u_in_past_3"] = oof_target.groupby("breath_id")["u_in"].shift(3)
# oof_target["u_in_past_4"] = oof_target.groupby("breath_id")["u_in"].shift(4)

# oof_target["u_in_diff_1"] = oof_target["u_in"] - oof_target["u_in_past_1"]
# oof_target["u_in_diff_2"] = oof_target["u_in"] - oof_target["u_in_past_2"]
# oof_target["u_in_diff_3"] = oof_target["u_in"] - oof_target["u_in_past_3"]
# oof_target["u_in_diff_4"] = oof_target["u_in"] - oof_target["u_in_past_4"]

# oof_target["u_in_cumsum"] = oof_target.groupby("breath_id")["u_in"].cumsum()

In [47]:
# v2
train_df['breath_id'] = train['breath_id']
train_df["pred_std"] = train_df[pred_cols].std(axis=1)
train_df["pred_max"] = train_df[pred_cols].values.max(axis=1)
train_df["pred_min"] = train_df[pred_cols].values.min(axis=1)
train_df["pred_max-min"] = train_df["pred_max"] - train_df["pred_min"]
train_df["pred_max-median"] = train_df["pred_max"] - train_df["pred_median"]
train_df["pred_max-mean"] = train_df["pred_max"] - train_df["pred_mean"]
train_df["pred_median-min"] = train_df["pred_median"] - train_df["pred_min"]
train_df["pred_mean-min"] = train_df["pred_mean"] - train_df["pred_min"]
train_df["pred_mean-median"] = train_df["pred_mean"] - train_df["pred_median"]
train_df["pred_kurt"] = train_df[pred_cols].kurt(axis=1)
for col_ in pred_cols:
    train_df[f"{col_}_past_1"] = train_df.groupby("breath_id")[f"{col_}"].shift(1)
    train_df[f"{col_}_past_2"] = train_df.groupby("breath_id")[f"{col_}"].shift(2)
    train_df[f"{col_}_past_3"] = train_df.groupby("breath_id")[f"{col_}"].shift(3)
    train_df[f"{col_}_past_4"] = train_df.groupby("breath_id")[f"{col_}"].shift(4)

    train_df[f"{col_}_diff_1"] = train_df[f"{col_}"] - train_df[f"{col_}_past_1"]
    train_df[f"{col_}_diff_2"] = train_df[f"{col_}"] - train_df[f"{col_}_past_2"]
    train_df[f"{col_}_diff_3"] = train_df[f"{col_}"] - train_df[f"{col_}_past_3"]
    train_df[f"{col_}_diff_4"] = train_df[f"{col_}"] - train_df[f"{col_}_past_4"]

train_df["u_in_past_1"] = train_df.groupby("breath_id")["u_in"].shift(1)
train_df["u_in_past_2"] = train_df.groupby("breath_id")["u_in"].shift(2)
train_df["u_in_past_3"] = train_df.groupby("breath_id")["u_in"].shift(3)
train_df["u_in_past_4"] = train_df.groupby("breath_id")["u_in"].shift(4)

train_df["u_in_diff_1"] = train_df["u_in"] - train_df["u_in_past_1"]
train_df["u_in_diff_2"] = train_df["u_in"] - train_df["u_in_past_2"]
train_df["u_in_diff_3"] = train_df["u_in"] - train_df["u_in_past_3"]
train_df["u_in_diff_4"] = train_df["u_in"] - train_df["u_in_past_4"]

train_df["u_in_cumsum"] = train_df.groupby("breath_id")["u_in"].cumsum()

del train_df['breath_id']

In [48]:
train_df

,C,R,time_step,u_in,u_out,pred_0,pred_1,pred_2,pred_3,pred_4,...,pred_7_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4,u_in_cumsum
0,50,20,0.000000,0.083334,0,5.756698,5.625724,5.696933,5.941694,5.814224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.083334
1,50,20,0.033652,18.383041,0,5.857983,5.831684,5.795729,5.846353,5.847313,...,NaN,0.083334,NaN,NaN,NaN,18.299707,NaN,NaN,NaN,18.466375
2,50,20,0.067514,22.509278,0,7.946752,7.917322,7.891675,8.084612,8.069213,...,NaN,18.383041,0.083334,NaN,NaN,4.126236,22.425944,NaN,NaN,40.975653
3,50,20,0.101542,22.808822,0,11.937506,12.020861,12.114049,12.239206,12.118088,...,NaN,22.509278,18.383041,0.083334,NaN,0.299544,4.425781,22.725488,NaN,63.784476
4,50,20,0.135756,25.355850,0,12.329441,12.381235,12.404347,12.477226,12.604493,...,6.731630,22.808822,22.509278,18.383041,0.083334,2.547028,2.846573,6.972809,25.272516,89.140326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,10,50,0.834147,1.869367,0,29.485389,29.470707,29.448645,29.463127,29.460402,...,NaN,2.650333,2.438287,3.783043,NaN,-0.780965,-0.568920,-1.913676,NaN,10.741030
2290964,10,50,0.867574,2.154414,0,29.157852,29.136485,29.126780,29.111472,29.111569,...,1.332623,1.869367,2.650333,2.438287,3.783043,0.285047,-0.495918,-0.283873,-1.628629,12.895445
2290965,10,50,0.900917,1.304434,0,29.897820,29.900912,29.885802,29.896856,29.892732,...,0.885935,2.154414,1.869367,2.650333,2.438287,-0.849980,-0.564933,-1.345899,-1.133853,14.199879
2290966,10,50,0.934309,1.733830,0,29.399083,29.391791,29.384078,29.391071,29.388413,...,0.620285,1.304434,2.154414,1.869367,2.650333,0.429396,-0.420585,-0.135538,-0.916503,15.933708


In [49]:
# train = oof_target.copy().reset_index(drop=True)

In [50]:
train = train.loc[train["u_out"] == 0].reset_index(drop=True)

In [51]:
train["R_C"] = train["R"].astype(str) + "_" + train["C"].astype(str)
train_gby = train.groupby("breath_id")["R_C"].agg("first").reset_index()
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed).split(train_gby, train_gby["R_C"])

fold_df = pd.DataFrame()
fold_df["id"] = train["id"]
fold_df["fold"] = -1

for fold, (train_idx, valid_idx) in enumerate(cv):
    valid_breath_ids = train_gby.iloc[valid_idx]["breath_id"].values
    idxs = train[train["breath_id"].isin(valid_breath_ids)].index.to_list()
    fold_df.loc[idxs, "fold"] = fold

In [52]:
y = train["pressure"]
X = train_df

In [53]:
X

,C,R,time_step,u_in,u_out,pred_0,pred_1,pred_2,pred_3,pred_4,...,pred_7_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4,u_in_cumsum
0,50,20,0.000000,0.083334,0,5.756698,5.625724,5.696933,5.941694,5.814224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.083334
1,50,20,0.033652,18.383041,0,5.857983,5.831684,5.795729,5.846353,5.847313,...,NaN,0.083334,NaN,NaN,NaN,18.299707,NaN,NaN,NaN,18.466375
2,50,20,0.067514,22.509278,0,7.946752,7.917322,7.891675,8.084612,8.069213,...,NaN,18.383041,0.083334,NaN,NaN,4.126236,22.425944,NaN,NaN,40.975653
3,50,20,0.101542,22.808822,0,11.937506,12.020861,12.114049,12.239206,12.118088,...,NaN,22.509278,18.383041,0.083334,NaN,0.299544,4.425781,22.725488,NaN,63.784476
4,50,20,0.135756,25.355850,0,12.329441,12.381235,12.404347,12.477226,12.604493,...,6.731630,22.808822,22.509278,18.383041,0.083334,2.547028,2.846573,6.972809,25.272516,89.140326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,10,50,0.834147,1.869367,0,29.485389,29.470707,29.448645,29.463127,29.460402,...,NaN,2.650333,2.438287,3.783043,NaN,-0.780965,-0.568920,-1.913676,NaN,10.741030
2290964,10,50,0.867574,2.154414,0,29.157852,29.136485,29.126780,29.111472,29.111569,...,1.332623,1.869367,2.650333,2.438287,3.783043,0.285047,-0.495918,-0.283873,-1.628629,12.895445
2290965,10,50,0.900917,1.304434,0,29.897820,29.900912,29.885802,29.896856,29.892732,...,0.885935,2.154414,1.869367,2.650333,2.438287,-0.849980,-0.564933,-1.345899,-1.133853,14.199879
2290966,10,50,0.934309,1.733830,0,29.399083,29.391791,29.384078,29.391071,29.388413,...,0.620285,1.304434,2.154414,1.869367,2.650333,0.429396,-0.420585,-0.135538,-0.916503,15.933708


In [54]:
X = X.fillna(0)

In [55]:
scaler = RobustScaler()
scaler.fit(X)

RobustScaler()

In [56]:
X = pd.DataFrame(scaler.transform(X.values), columns=list(X))

In [57]:
import lightgbm as lgb

models = []
oof_prediction = np.zeros(len(X))

for fold in range(n_fold):
    print("---------------------------------")
    print(f"Fold {fold}")
    print("---------------------------------")

    train_idx = fold_df[fold_df["fold"] != fold].index
    valid_idx = fold_df[fold_df["fold"] == fold].index

    train_X, train_y = X.iloc[train_idx], y.iloc[train_idx]
    valid_X, valid_y = X.iloc[valid_idx], y.iloc[valid_idx]
    
    model = RidgeCV(alphas=np.logspace(-3,10, 20))
    model.fit(train_X, train_y)

#     lgb_train = lgb.Dataset(train_X, train_y)
#     lgb_valid = lgb.Dataset(valid_X, valid_y, reference=lgb_train)

#     # LightGBM parameters
#     params = {
#             'boosting_type': 'gbdt',
#             'objective': 'regression_l1',
#             'metric': 'l1',
#             'seed': seed,
#             'num_boost_round': 10000,
#             'learning_rate': 0.1,
#             'lambda_l1': 0.1,
#             'lambda_l2': 0.1,
#     }


#     model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_valid, early_stopping_rounds=200, verbose_eval=100)
    models.append(model)

    oof_prediction[valid_idx] = model.predict(valid_X)
    score = np.abs(valid_y.values - oof_prediction[valid_idx]).mean()
    print(f'fold = {fold}, score = {score}')

---------------------------------
Fold 0
---------------------------------
fold = 0, score = 0.11686268716652391
---------------------------------
Fold 1
---------------------------------
fold = 1, score = 0.11957006001998888
---------------------------------
Fold 2
---------------------------------
fold = 2, score = 0.11706003581063425
---------------------------------
Fold 3
---------------------------------
fold = 3, score = 0.11914011084536763
---------------------------------
Fold 4
---------------------------------
fold = 4, score = 0.12160959861698038
---------------------------------
Fold 5
---------------------------------
fold = 5, score = 0.12327238031023417
---------------------------------
Fold 6
---------------------------------
fold = 6, score = 0.11721902306334239
---------------------------------
Fold 7
---------------------------------
fold = 7, score = 0.11682257425810481
---------------------------------
Fold 8
---------------------------------
fold = 8, score = 0.1

In [58]:
CV = np.abs(y.values - oof_prediction).mean()
CV

0.11915307598079279

In [ ]:
# import os
# import pickle

# os.makedirs(f"./v{version}", exist_ok=True)

# for fold, model in enumerate(models):
#     with open(f"./v{version}/fold-{fold}.pickle", "wb") as f:
#         pickle.dump(model, f)

# np.save(f"./v{version}/{name}_stacking_v{version}_oof_cv{CV:.4f}.npy", oof_prediction)